In [ ]:
import formulas
import utilities
import pandas as pd
import os
import re


def create_formulas_content(website_creator_dir, docs_dir):

    formula_file_path = website_creator_dir + os.path.sep + 'formulas.csv' 
    order_file_path = website_creator_dir + os.path.sep + 'sort_orders.csv'
    
    df_by_year =  (
        formulas.get_formulas_by_year_df(formula_file_path))
    df_cumulative = (
        formulas.get_formulas_by_year_cumulative_df(
            formula_file_path, order_file_path))
    sort_orders_df = pd.read_csv(order_file_path)
    df_formulas = pd.concat([df_by_year, df_cumulative])
    
    dirs_df = (
        df_formulas[['State', 'Formula sub category 1', 
                             'Formula sub category 2',
                             'Subject code']].drop_duplicates())
    file_paths_df = (
        df_formulas[['State', 'Formula sub category 1', 
                             'Formula sub category 2',
                             'Subject code', 'Category']].drop_duplicates())
    
    utilities.create_sub_directories_from_df(base_dir = docs_dir, 
                                             sub_paths_df = dirs_df)
    
    front_matter_index_files = {'bookCollapseSection' : True}
    utilities.create_index_files(base_dir=docs_dir, dirs_df=dirs_df, 
                       front_matter=front_matter_index_files,
                       sort_orders_df = sort_orders_df)
    
    utilities.create_files(base_dir = docs_dir, file_paths_df= file_paths_df, 
                           file_extension='.md', 
                           fn=formulas.get_formula_display_string, 
                           sort_orders_df = sort_orders_df,
                           df_formulas = df_formulas)


if __name__ == '__main__':

    website_creator_dir = os.getcwd()
    docs_dir = utilities.get_docs_path(website_creator_dir)
    utilities.delete_directory_if_it_exists(docs_dir)
    create_formulas_content(website_creator_dir, docs_dir)


    # df_calculus = formulas.df_calculus_summary(df_formulas)
    # styler_calculus = formulas.styler_calculus_summary(
    #     df_calculus, formula_sheet_list)
    # display(styler_calculus)

In [12]:
import formulas
import utilities
import pandas as pd
import os
import re





def get_formula_display_string(input_series, **kwarg):
    """Returns a summmary formula table in markdown format with embedded
    html.  Input series is a pandas series with Indices State, Subec code
    and category.  **Kwarg must be called with parameter  = df_formulas
    where df_Formulas contains fields State, Subject code, Category, 
    Formula_1, Formula_2"""
    
    df_formulas = kwarg['df_formulas']
    df = df_formulas[(
        (df_formulas['State'] == str(input_series['State'])) &
        (df_formulas['Formula sub category 2'] == str(input_series['Formula sub category 2'])) &
        (df_formulas['Subject code'] == str(input_series['Subject code'])) & 
        (df_formulas['Category'] == str(input_series['Category'])))]
        
    formula_sheet_list =kwarg.get('formula_sheet_list')

    cols_to_highlight_if_in_formula_sheet = (
        kwarg.get('cols_to_highlight_if_in_formula_sheet'))
    
    formula_2_col_is_empty = df['Formula_2'].dropna().empty    
    if formula_2_col_is_empty:
        df = df[['Formula_1']]
    else:
        df = df[['Formula_1', 'Formula_2']]
    
    formula_set_styler = (formulas.df_to_formula_styled_table(
        df=df, col_widths={'Formula_1':300, 'Formula_2':400},
        cols_to_highlight_if_in_formula_sheet = (
            cols_to_highlight_if_in_formula_sheet),
        formula_sheet_list = formula_sheet_list,
        display_col_headers = False))
    output_string =  '#  \n<br>\n' + formula_set_styler.to_html()

    ### UNCOMMENT THE RETURN VALUE AND DELETE THE TEMP RETURN !!!!! ##
    # return(output_string)
    return(formula_set_styler)


if __name__ == '__main__':

    website_creator_dir = os.getcwd()
    docs_dir = utilities.get_docs_path(website_creator_dir)
    formula_file_path = website_creator_dir + os.path.sep + 'formulas.csv'

    df_formulas = pd.read_csv(formula_file_path)
    df_formulas['Formula sub category 2'] = 'By Year'

    input_series = pd.Series(
        ['NSW','By Year', 'Year 12 Adv', 'Integration'], 
        index = ['State','Formula sub category 2', 
                 'Subject code', 'Category' ])

    formula_sheet_list = formulas.get_formulas_on_formula_sheet(formula_file_path)
    
    display_string = get_formula_display_string(
        input_series = input_series, 
        df_formulas = df_formulas,
        cols_to_highlight_if_in_formula_sheet = ['Formula_1', 'Formula_2'],
        formula_sheet_list = formula_sheet_list)

    display(display_string)

    # df_calculus = formulas.df_calculus_summary(df_formulas)
    # styler_calculus = formulas.styler_calculus_summary(
    #     df_calculus, formula_sheet_list)
    # display(styler_calculus)
        

$ {\Large\int} f'(x)[f(x)]^n dx = \dfrac{1}{n+1}[f(x)]^{n+1} + c $ $ \text{ where } n \neq -1 $
$ {\Large\int} u \dfrac{dv}{dx} dx=uv-{\Large\int}v \dfrac {du}{dx}dx$
$ {\Large\int_{a}^{b}} f(x) dx \approx \dfrac{b-a} {2n} {\Large\{} f(a) + f(b) + 2 {\Large[} f(x_1)+...+f(x_{n-1}){\Large ]} {\Large\}}$ $ \text { where } a=x_0 \text{ and } b=x_n $
$ {\Large\int} f'(x)cosf(x)dx = sin f(x) + c$
$ {\Large\int} f'(x)sin f(x)dx = -cos f(x) + c$
$ {\Large\int} f'(x)sec^2 f(x)dx = tan f(x) + c$
$ {\Large\int} f'(x)e^{f(x)}dx = e^{f(x)} + c$
$ {\Large\int} \dfrac{f'(x)}{f(x)}dx = \ln|f(x)| + c$
$ {\Large\int} f'(x)a^{f(x)}dx = \dfrac{a^f(x)}{\ln a} +c$
$ {\Large\int} \dfrac{f'(x)}{(\ln a) f(x)} = \log_{a} |f(x)| + c$
